In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense
from keras.optimizers import Adam


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mauricebaier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mauricebaier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
fake = pd.read_csv('Fake.csv')
fake.head()
#fake.isna().sum()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true = pd.read_csv('True.csv')
true.head()
#true.isna().sum()


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
def text_preprocessing(text):
    text = text.lower()
    tokens = word_tokenize(text)

    new_text = []
    ps = PorterStemmer()

    for w in tokens:
        if w not in stopwords.words('english') and len(w) > 2:
            stem_token = ps.stem(w)   
            new_text.append(stem_token)  # Add the stemmed word to new_tokens
    
    text_prep = ' '.join(new_text)  # Join stemmed tokens into a string
    return text_prep

In [7]:
fake['text'] = fake['text'].apply(text_preprocessing)
true['text'] = true['text'].apply(text_preprocessing)


In [8]:
fake['true'] = 0
true['true'] = 1
data = pd.concat([fake, true], axis=0)
data.tail()

,title,text,subject,date,true
21412,'Fully committed' NATO backs new U.S. approach...,brussel reuter nato alli tuesday welcom presid...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,london reuter lexisnexi provid legal regulator...,worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,minsk reuter shadow disus soviet-era factori m...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,moscow reuter vatican secretari state cardin p...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,jakarta reuter indonesia buy sukhoi fighter je...,worldnews,"August 22, 2017",1


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding  # Import Embedding layer
from tensorflow.keras.optimizers import Adam
from nltk.corpus import stopwords


X = data['text']
y = data['true']

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to ensure uniform length
max_len = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Define Bi-LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100))
model.add(Bidirectional(LSTM(units=64, activation='relu')))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Model Accuracy:", accuracy)

Epoch 1/2
449/449 ━━━━━━━━━━━━━━━━━━━━ 2353s 5s/step - accuracy: 0.5289 - loss: nan - val_accuracy: 0.5267 - val_loss: nan
Epoch 2/2
449/449 ━━━━━━━━━━━━━━━━━━━━ 2420s 5s/step - accuracy: 0.5178 - loss: nan - val_accuracy: 0.5267 - val_loss: nan
281/281 ━━━━━━━━━━━━━━━━━━━━ 125s 445ms/step - accuracy: 0.5189 - loss: nan
Model Accuracy: 0.5270601511001587


In [47]:
test_texts = data['text'].sample(100)

# Tokenize and pad the sequences
sequences = tokenizer.texts_to_sequences(test_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Predict the labels
predictions = model.predict(padded_sequences)

# Convert the predicted probabilities to labels
predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]
print(predictions) 

# Print the text, predicted label, and actual label
correct_predictions = 0
for i in range(len(test_texts)):
    print("Text:", test_texts.iloc[i])
    print("Predicted Label:", predicted_labels[i])
    print("Actual Label:", data.loc[X_test[:, 0]]['true'].iloc[i])
    print()
    
    if predicted_labels[i] == data.loc[X_test[:, 0]]['true'].iloc[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(test_texts)
print("Accuracy:", accuracy)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Text: washington reuter top white hous offici met moder conserv republican u.s. hous repres monday effort reviv plan repeal replac obamacar key member administr includ vice presid mike penc invit group moder republican known tuesday group white hous penc went 